In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,33983.58,-0.23,0.37,0.55,0.07,0.08,0.33,0.64,-0.03,0.12,...,0.01,0.01,-0.01,0.03,0.01,0.00,0.00,-0.00,0.00,-0.00
3,-0.23,23455.66,0.03,-0.39,0.33,-0.54,0.08,0.14,-0.08,-0.05,...,-0.01,0.02,0.00,0.01,0.01,-0.00,0.00,-0.00,-0.00,-0.00
4,0.37,0.03,17439.68,0.58,0.20,-0.03,0.21,-0.21,0.24,0.00,...,-0.04,0.00,0.01,-0.01,0.00,0.00,0.00,-0.00,0.00,0.00
5,0.55,-0.39,0.58,12267.48,-0.43,0.39,-0.03,-0.11,0.03,0.02,...,-0.01,-0.01,0.00,-0.00,0.01,0.00,0.00,-0.00,0.00,0.00
6,0.07,0.33,0.20,-0.43,8044.52,-0.01,-0.02,-0.01,-0.11,0.06,...,0.01,-0.00,0.01,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
7,0.08,-0.54,-0.03,0.39,-0.01,4897.66,-0.01,0.01,0.05,-0.02,...,-0.02,0.01,-0.01,0.01,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.33,0.08,0.21,-0.03,-0.02,-0.01,3008.94,0.02,-0.09,0.07,...,0.00,0.01,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
9,0.64,0.14,-0.21,-0.11,-0.01,0.01,0.02,1688.27,-0.04,-0.02,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
10,-0.03,-0.08,0.24,0.03,-0.11,0.05,-0.09,-0.04,1016.96,0.04,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00
11,0.12,-0.05,0.00,0.02,0.06,-0.02,0.07,-0.02,0.04,601.69,...,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00002,0.00003,0.00000,0.00001,0.00003,0.00008,-0.00001,0.00003,...,0.00001,0.00001,-0.00001,0.00004,0.00003,0.00000,0.00001,-0.00000,0.00001,-0.00000
3,-0.00001,1.00000,0.00000,-0.00002,0.00002,-0.00005,0.00001,0.00002,-0.00002,-0.00001,...,-0.00001,0.00002,0.00000,0.00002,0.00001,-0.00001,0.00001,-0.00002,-0.00001,-0.00000
4,0.00002,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00006,0.00000,...,-0.00003,0.00000,0.00002,-0.00002,0.00000,0.00001,0.00000,-0.00001,0.00001,0.00000
5,0.00003,-0.00002,0.00004,1.00000,-0.00004,0.00005,-0.00000,-0.00002,0.00001,0.00001,...,-0.00001,-0.00002,0.00001,-0.00001,0.00002,0.00000,0.00000,-0.00001,0.00001,0.00001
6,0.00000,0.00002,0.00002,-0.00004,1.00000,-0.00000,-0.00000,-0.00000,-0.00004,0.00003,...,0.00001,-0.00000,0.00001,0.00000,0.00001,0.00002,-0.00001,0.00001,-0.00001,-0.00000
7,0.00001,-0.00005,-0.00000,0.00005,-0.00000,1.00000,-0.00000,0.00000,0.00002,-0.00001,...,-0.00003,0.00001,-0.00002,0.00005,-0.00001,0.00002,0.00001,-0.00002,0.00000,-0.00001
8,0.00003,0.00001,0.00003,-0.00000,-0.00000,-0.00000,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00002,-0.00001,0.00000,-0.00000,0.00000,-0.00001,-0.00003,-0.00001,-0.00000
9,0.00008,0.00002,-0.00004,-0.00002,-0.00000,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,-0.00000,-0.00001,0.00003,-0.00001,0.00001,0.00000,0.00001,-0.00001,-0.00002
10,-0.00001,-0.00002,0.00006,0.00001,-0.00004,0.00002,-0.00005,-0.00003,1.00000,0.00005,...,0.00001,-0.00000,0.00000,-0.00001,0.00001,0.00002,-0.00001,0.00001,0.00001,0.00001
11,0.00003,-0.00001,0.00000,0.00001,0.00003,-0.00001,0.00005,-0.00002,0.00005,1.00000,...,0.00001,0.00001,0.00004,-0.00001,0.00003,0.00003,-0.00002,-0.00001,-0.00002,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

2.2888013006891708

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.03620485e+07 2.39271880e+07 1.31680353e+07 6.49427813e+06
 2.78484683e+06 1.02983027e+06 3.87177279e+05 1.21583469e+05
 4.39030318e+04 1.52990820e+04 5.55491805e+03 2.04582791e+03
 7.62542838e+02 2.57830203e+02 8.33375820e+01 2.34901032e+01
 6.24550506e+00 1.19093417e+00 2.25748617e-01 3.45871388e-02
 7.42050719e-03 1.41037283e-03 2.52902475e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997632,-0.057936,-0.029860,-0.017620,-0.010618,-0.006219,-0.003761,-0.002084,-0.001266,-0.000745,...,-0.000098,-0.000056,-0.000030,-0.000015,-0.000007,-0.000003,-0.000001,-6.081987e-07,-2.706822e-07,-1.165813e-07
1,0.054530,0.994601,-0.079088,-0.032932,-0.017540,-0.009855,-0.005859,-0.003241,-0.001956,-0.001155,...,-0.000152,-0.000086,-0.000047,-0.000024,-0.000010,-0.000005,-0.000002,-9.855869e-07,-4.973429e-07,-1.742416e-07
2,0.032304,0.074087,0.993488,-0.072391,-0.029393,-0.015174,-0.008806,-0.004854,-0.002884,-0.001712,...,-0.000227,-0.000129,-0.000068,-0.000037,-0.000016,-0.000007,-0.000003,-1.402973e-06,-6.596780e-07,-2.539583e-07
3,0.020714,0.035599,0.066680,0.994425,-0.063663,-0.025388,-0.013887,-0.007456,-0.004423,-0.002606,...,-0.000344,-0.000197,-0.000105,-0.000055,-0.000024,-0.000011,-0.000004,-2.142216e-06,-9.670918e-07,-3.675240e-07
4,0.013255,0.020447,0.030611,0.058756,0.995509,-0.056049,-0.024588,-0.012332,-0.007218,-0.004207,...,-0.000552,-0.000317,-0.000168,-0.000088,-0.000039,-0.000017,-0.000007,-3.280807e-06,-1.740255e-06,-6.292413e-07
5,0.008120,0.012061,0.016767,0.025919,0.051764,0.995889,-0.059867,-0.023065,-0.012713,-0.007310,...,-0.000953,-0.000541,-0.000292,-0.000149,-0.000067,-0.000029,-0.000012,-5.934312e-06,-2.861358e-06,-1.091936e-06
6,0.005249,0.007679,0.010448,0.015328,0.025395,0.056166,0.996059,-0.052820,-0.024147,-0.013059,...,-0.001653,-0.000943,-0.000507,-0.000264,-0.000117,-0.000052,-0.000021,-1.042301e-05,-5.093050e-06,-1.859864e-06
7,0.003032,0.004427,0.005999,0.008597,0.013418,0.023776,0.048790,0.995766,-0.064213,-0.026939,...,-0.003075,-0.001758,-0.000941,-0.000487,-0.000217,-0.000096,-0.000038,-1.852479e-05,-9.420978e-06,-3.571927e-06
8,0.001968,0.002858,0.003831,0.005481,0.008451,0.014206,0.024911,0.059944,0.994880,-0.066492,...,-0.005526,-0.003150,-0.001683,-0.000878,-0.000387,-0.000171,-0.000069,-3.321624e-05,-1.655676e-05,-6.135254e-06
9,0.001234,0.001796,0.002413,0.003432,0.005246,0.008710,0.014486,0.028258,0.061189,0.993986,...,-0.010097,-0.005705,-0.003032,-0.001584,-0.000696,-0.000308,-0.000125,-6.042795e-05,-3.048920e-05,-1.114867e-05


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0023683354335242157,
 0.005398877158835247,
 0.006511732978186191,
 0.005575003037449622,
 0.004490669986519813,
 0.004111018680279743,
 0.003940998781258975,
 0.004233790376154345,
 0.005119776815711075,
 0.0060136597519635515,
 0.007344423136560363,
 0.008837536530604151,
 0.009773175561432645,
 0.010351136502610991,
 0.010835638213344656,
 0.01145474631786314,
 0.011299569706129109,
 0.011970334618894962,
 0.015186467075250065,
 0.030105163729460482,
 0.06652432258188201,
 0.11462838078398985,
 0.08023486362833598]